#### CRUD operations:


**TODO 1**: You will continue to work on the MCU network for a while. Who are the 10 fastest in the MCU ? Are they related to each other ?

**TODO 2**: How many have collaborated atleast more than 10 times with another superhero?

**TODO 3**: Did the weakest of the MCU collaborated with the strongest ?

**TODO 4**: Give a stats that interests you !

**TODO 5**: Create a new db called "master". Create a graph from the M1 class with name and random age between 20 and 30, Group of 3 that collab (undirected relation).

**TODO 6**: Get the age for people YOU collaborate with.

**TODO 7**: Get nodes with age > 25.

### Real world problem:

**TODO 8**: Using the downloaded data in TODO 11 of chapter II, create a collaboration network between authors for papers after 2021 (same author if same name). Insert it in neo4j in a db called "arxiv".

**TODO 9**: Get the density of degrees of the collaboration network

**TODO 10**: Do a for loop for i in range(500) for each i remove nodes with a degree less than i.


In [ ]:
#1

import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")

query = """MATCH (n:Superhero)
WHERE (n.Speed) IS NOT NULL
RETURN n.name, n.Speed
ORDER BY n.Speed DESC
LIMIT 10"""

results = graph.run(query)

for record in results:
    print(record)

import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")

query = """MATCH (n:Superhero)-[r]-(m:Superhero)
WHERE n.name IN ['Black Bolt', 'Stardust', 'Quicksilver', 'Nova', 'Thor', 'Silver Surfer', 'Galactus', 'Dormammu', 'Ego', 'Northstar']
AND m.name IN ['Black Bolt', 'Stardust', 'Quicksilver', 'Nova', 'Thor', 'Silver Surfer', 'Galactus', 'Dormammu', 'Ego', 'Northstar']
RETURN n.name, m.name"""

results = graph.run(query)

for record in results:
    print(record)


#2

import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")

query = """MATCH (a:Superhero)-[r:RELATION]-(b:Superhero)
WHERE r.n_collab >= 10
WITH a
MATCH (a)-[r:RELATION]-(b:Superhero)
WHERE r.n_collab >= 10
RETURN COUNT(DISTINCT a)"""

result = graph.run(query).evaluate()
print(f"{result} super héros ont collaboré plus de 10 fois avec un autre super héro")

#3

import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")

query_weakest = """MATCH (n:Superhero)
WHERE (n.Total) IS NOT NULL
RETURN n.name
ORDER BY n.Total
LIMIT 1"""
result_weakest = graph.run(query_weakest).evaluate()

query_strongest = """MATCH (n:Superhero)
WHERE (n.Total) IS NOT NULL
RETURN n.name
ORDER BY n.Total DESC
LIMIT 1"""
result_strongest = graph.run(query_strongest).evaluate()

query_relation = f"""MATCH (a:Superhero {{name: "{result_weakest}"}})-[r:RELATION]-(b:Superhero {{name: "{result_strongest}"}})
RETURN COUNT(r)"""
result_relation = graph.run(query_relation).evaluate()

if result_relation > 0:
    print(f"{result_weakest} a collaboré avec {result_strongest}")
else:
    print(f"{result_weakest} n'a pas collaboré avec {result_strongest}")

#4


#5

import py2neo
import random

graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="master")

noms = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
personnes = [py2neo.Node("M1", name=nom, age=random.randint(20, 30)) for nom in noms]

for personne in personnes:
    graph.create(personne)

groupes = []
while len(personnes) >= 3:
    groupe = random.sample(personnes, 3)
    groupes.append(groupe)
    for personne in groupe:
        personnes.remove(personne)

for groupe in groupes:
    for i, j in zip(groupe, groupe[1:] + [groupe[0]]):
        relation = py2neo.Relationship(i, "collab", j)
        graph.create(relation)


#6

import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="master")

query = """MATCH (a:M1 {name:"A"})-[:collab]-(b:M1)
RETURN b.name, b.age"""

results = graph.run(query)
print(results)

#7

import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="master")

query = """MATCH (a:M1)
WHERE a.age > 25
RETURN a.name, a.age"""

df = graph.run(query).to_data_frame()
print(df)


#8


#9


#10

